In [25]:
!pip install langchain
!pip install langchain_google_genai
!pip install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.5 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.9 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.17
    Uninstalling google-ai-generativelanguage-0.6.17:
      Successfully uninstalled google-ai-generativelanguage-0.6.17
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.2 requires google-ai-generativelanguage<0.7.0,>=0.6.16, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [27]:
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.document_loaders import WikipediaLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.generativeai import configure

# Replace with your Google Gemini API key
GOOGLE_API_KEY = "your api here!!!"

# Configure Google Gemini AI
configure(api_key=GOOGLE_API_KEY)

# Function to create a Wikipedia-based retriever
def create_wikipedia_retriever(query):
    # Load Wikipedia documents
    loader = WikipediaLoader(query=query, lang="en")
    docs = loader.load()

    # Split documents into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(docs)

    # Generate embeddings using Google AI
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

    # Create a FAISS vector store
    vectorstore = FAISS.from_documents(split_docs, embeddings)

    # Return FAISS retriever (use as_retriever())
    return vectorstore.as_retriever()

# Initialize Gemini AI LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY)

# Function to get a response from Wikipedia-based chatbot
def get_response(user_query):
    retriever = create_wikipedia_retriever(user_query)  # Create retriever based on query
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)  # Setup RetrievalQA chain
    response = qa_chain.run(user_query)  # Get response
    return response

# Simple terminal-based chatbot
print("Wikipedia Chatbot (Gemini AI)")
print("Type 'exit' to quit.\n")

while True:
    user_query = input("You: ")
    if user_query.lower() == "exit":
        print("Goodbye!")
        break
    response = get_response(user_query)
    print("Bot:", response)

Wikipedia Chatbot (Gemini AI)
Type 'exit' to quit.



KeyboardInterrupt: Interrupted by user

In [33]:
import requests
from bs4 import BeautifulSoup
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings  # Switched to free embeddings
import os

# Define available sources
AVAILABLE_SOURCES = {
    "wikipedia": "https://en.wikipedia.org/wiki/",
    "arxiv": "https://arxiv.org/search/?query=",
    "bbc_news": "https://www.bbc.co.uk/search?q=",
    "nature": "https://www.nature.com/search?q=",
    "custom": ""  # For user-defined URLs
}

# Fetch content from Wikipedia
def fetch_wikipedia(query):
    url = AVAILABLE_SOURCES["wikipedia"] + query.replace(" ", "_")
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        paragraphs = soup.find_all("p", limit=5)
        return "\n".join([p.get_text() for p in paragraphs]) or "No relevant content found."
    except requests.RequestException as e:
        return f"Error fetching Wikipedia content: {e}"

# Fetch research content from ArXiv or Nature
def fetch_research_papers(query, source):
    if source == "arxiv":
        url = AVAILABLE_SOURCES["arxiv"] + query.replace(" ", "+")
    elif source == "nature":
        url = AVAILABLE_SOURCES["nature"] + query.replace(" ", "+")
    else:
        return "Invalid research source."
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        if source == "arxiv":
            abstracts = soup.select("span.abstract-full", limit=5)
            return "\n".join([a.get_text().strip() for a in abstracts]) or "No relevant content found."
        else:  # Nature
            paragraphs = soup.find_all("p", limit=5)
            return "\n".join([p.get_text() for p in paragraphs]) or "No relevant content found."
    except requests.RequestException as e:
        return f"Error fetching content from {source}: {e}"

# Fetch content from a custom URL
def fetch_custom(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        paragraphs = soup.find_all("p", limit=5)
        return "\n".join([p.get_text() for p in paragraphs]) or "No relevant content found."
    except requests.RequestException as e:
        return f"Error fetching content from {url}: {e}"

# Initialize LLM with OpenRouter, adding required headers
llm = ChatOpenAI(
    openai_api_base="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY", "your api here"
),
    model="mistralai/mixtral-8x7b-instruct",  # Replace with correct model from OpenRouter
    temperature=0.5,
    default_headers={
        "HTTP-Referer": "http://localhost",  # Required by OpenRouter
        "X-Title": "RAG Chatbot"  # Required by OpenRouter
    }
)

# Initialize ChromaDB with free Hugging Face embeddings (no OpenAI key needed)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma(collection_name="dynamic_data", embedding_function=embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Define prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="Based on the following information:\n{context}\nAnswer this question: {question}"
)

# Set up RetrievalQA chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# Store content in vector database
def store_content(query, content, source):
    if "Error" not in content:
        vectorstore.add_texts(
            texts=[content],
            metadatas=[{"query": query, "source": source}]
        )

# RAG Chatbot function
def rag_chatbot(query, source):
    if source == "wikipedia":
        content = fetch_wikipedia(query)
    elif source in ["arxiv", "nature"]:
        content = fetch_research_papers(query, source)
    elif source.startswith("http"):
        content = fetch_custom(source)
    else:
        return "Invalid source. Please select Wikipedia, ArXiv, Nature, or provide a valid URL."

    store_content(query, content, source)
    response = rag_chain({"query": query})
    return response["result"]

# Test the chatbot
query = "Madras institute of technology"
source = "wikipedia"
response = rag_chatbot(query, source)
print(response)

 Based on the information provided, Madras Institute of Technology (MIT) is a college that is part of Anna University. It is located in Chromepet Campus, Chennai, India. MIT was established in 1949 and became a part of Anna University in 1978. It was originally a private self-financing institution but is now a government-aided college.


In [35]:
# Install required packages
!pip install langchain langchain-openai faiss-cpu requests sentence-transformers beautifulsoup4 chromadb -q

# Import libraries
import requests
from bs4 import BeautifulSoup
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define available sources
AVAILABLE_SOURCES = {
    "wikipedia": "https://en.wikipedia.org/wiki/",
    "arxiv": "https://arxiv.org/search/?query=",
    "bbc_news": "https://www.bbc.co.uk/search?q=",
    "nature": "https://www.nature.com/search?q="
}

# Fetch content from Wikipedia
def fetch_wikipedia(query):
    url = AVAILABLE_SOURCES["wikipedia"] + query.replace(" ", "_")
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        content_div = soup.find("div", {"id": "mw-content-text"})
        paragraphs = content_div.find_all("p", limit=5) if content_div else []
        content = "\n".join([p.get_text().strip() for p in paragraphs])
        return content if content else "No relevant content found on Wikipedia."
    except requests.RequestException as e:
        return f"Error fetching Wikipedia content: {str(e)}"

# Fetch research content from ArXiv or Nature
def fetch_research_papers(query, source):
    if source not in ["arxiv", "nature"]:
        return "Invalid research source. Use 'arxiv' or 'nature'."
    url = AVAILABLE_SOURCES[source] + query.replace(" ", "+")
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        if source == "arxiv":
            abstracts = soup.select("span.abstract-full", limit=5)
            content = "\n".join([a.get_text().strip() for a in abstracts])
        else:  # Nature
            articles = soup.find_all("article", limit=5)
            content = "\n".join([a.get_text().strip() for a in articles]) if articles else ""
        return content if content else f"No relevant content found on {source}."
    except requests.RequestException as e:
        return f"Error fetching content from {source}: {str(e)}"

# Fetch content from BBC News
def fetch_bbc_news(query):
    url = AVAILABLE_SOURCES["bbc_news"] + query.replace(" ", "+")
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        results = soup.find_all("div", {"data-testid": "default-promo"}, limit=5)
        content = "\n".join([r.get_text().strip() for r in results])
        return content if content else "No relevant content found on BBC News."
    except requests.RequestException as e:
        return f"Error fetching BBC News content: {str(e)}"

# Fetch content from a custom URL
def fetch_custom(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        paragraphs = soup.find_all("p", limit=5)
        content = "\n".join([p.get_text().strip() for p in paragraphs])
        return content if content else "No relevant content found at the provided URL."
    except requests.RequestException as e:
        return f"Error fetching content from {url}: {str(e)}"

# Prompt user for OpenRouter API key

api_key = "your api here"
# Initialize LLM with OpenRouter
llm = ChatOpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1",
    model="mistralai/mixtral-8x7b-instruct",
    temperature=0.5,
    default_headers={
        "HTTP-Referer": "http://localhost",
        "X-Title": "RAG Chatbot"
    }
)

# Initialize ChromaDB with Hugging Face embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma(collection_name="dynamic_data", embedding_function=embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Define prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="Based on the following information:\n{context}\nAnswer this question: {question}"
)

# Set up RetrievalQA chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# Store content in vector database
def store_content(query, content, source):
    if "Error" not in content and "No relevant content" not in content:
        # Split content into chunks for better retrieval
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        docs = text_splitter.split_text(content)
        if docs:
            vectorstore.add_texts(
                texts=docs,
                metadatas=[{"query": query, "source": source}] * len(docs)
            )

# RAG Chatbot function
def rag_chatbot(query, source="wikipedia"):
    if source == "wikipedia":
        content = fetch_wikipedia(query)
    elif source in ["arxiv", "nature"]:
        content = fetch_research_papers(query, source)
    elif source == "bbc_news":
        content = fetch_bbc_news(query)
    elif source.startswith("http"):
        content = fetch_custom(source)
    else:
        return "Invalid source. Use 'wikipedia', 'arxiv', 'nature', 'bbc_news', or a valid URL."

    store_content(query, content, source)
    response = rag_chain.invoke({"query": query})
    return response["result"]

# Chat interface
def chat():
    print("Welcome to the Multi-Source RAG Chatbot!")
    print("Sources: wikipedia, arxiv, nature, bbc_news, or a custom URL")
    print("Type 'exit' to quit.\n")
    while True:
        query = input("You (query): ")
        if query.lower() == "exit":
            print("Goodbye!")
            break
        source = input("Source (default: wikipedia): ") or "wikipedia"
        try:
            response = rag_chatbot(query, source)
            print(f"Bot: {response}\n")
        except Exception as e:
            print(f"Error: {str(e)}\n")

# Start chatting
chat()

Welcome to the Multi-Source RAG Chatbot!
Sources: wikipedia, arxiv, nature, bbc_news, or a custom URL
Type 'exit' to quit.

You (query): Black Holes
Source (default: wikipedia): wikipedia
Bot:  User's question: How can black holes be detected if they don't emit light?

Black holes cannot be detected directly because they do not emit light, as explained in the context. However, their presence can be inferred through their interaction with other matter and with electromagnetic radiation such as visible light. Here are a few ways black holes are typically detected:

1. Accretion disk: Matter falling toward a black hole can form an accretion disk of infalling plasma, heated by friction and emitting light. In extreme cases, this creates a quasar, one of the brightest objects in the universe.

2. Shredded stars: Stars passing too close to a supermassive black hole can be shredded into streamers that shine very brightly before being "swallowed."

3. Orbiting objects: If other stars are orbiti

KeyboardInterrupt: Interrupted by user